## Assignment 5 - Group 7

### First, we install all packages necesary and the chrome driver 

In [1]:
# Links 
# chromedriver= win 64: https://googlechromelabs.github.io/chrome-for-testing/
# chromedriver= win 32 :  https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/120.0.6099.109/win32/chromedriver-win32.zip

In [3]:
# pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
# pip install webdriver_manager==3.4.2

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Now, we import each library and install each package necesary to do the web scrapping
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import time 
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

In [6]:
def scrape_page(i):
    # Create a webdriver instance for each thread
    driver = webdriver.Chrome()
    driver.maximize_window()

    try:
        # Open the specified URL in the Chrome browser
        driver.get(url)

        dropdown = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
        dropdown.click()
        time.sleep(2)

        elec_presidenciales = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
        elec_presidenciales.click()
        time.sleep(2)

        año_eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
        año_eleccion.click()
        time.sleep(1)

        año_eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{}]'.format(i))
        año_eleccion.click()
        año_eleccion_tipo = año_eleccion.text
        time.sleep(1)

        selecciona = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
        selecciona.click()
        time.sleep(1)

        pag_final = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
        pag_final.click()
        time.sleep(2)
    
        # Obtain page's HTML
        page_source = driver.page_source
    
        # Use BeautifulSoup to analize HTML
        soup = BeautifulSoup(page_source, 'html.parser')
    
        # Find the table
        data_table = soup.find('table', {'class': 'table'})

        # Extract data from table and store it
        table_data = []
        for row in data_table.find_all('tr'):
            row_data = [cell.text for cell in row.find_all('td')]
            table_data.append(row_data)

        # Return the scraped data
        return table_data

    finally:
        # Close the webdriver after usage
        driver.quit()

if __name__ == "__main__":
    url = 'https://infogob.jne.gob.pe/Eleccion'
    all_table_data = []

    with ThreadPoolExecutor(max_workers=5) as executor:
        # Use ThreadPoolExecutor to run scrape_page function for each value of i concurrently
        results = executor.map(scrape_page, range(2, 19))

    
    # After the loop, create a DataFrame from all_table_data
    flat_table_data = [item for sublist in results for item in sublist]

    # Create the DataFrame
    column_names = ['Organización Política', 'Símbolo', 'Plan de Gobierno', 'Total Votos', 'Porcentaje Votos Válidos', 'Lista de Candidatos']
    df = pd.DataFrame(flat_table_data, columns=column_names)

In [7]:
df

,Organización Política,Símbolo,Plan de Gobierno,Total Votos,Porcentaje Votos Válidos,Lista de Candidatos
0,None,None,None,None,None,None
1,PARTIDO POLITICO NACIONAL PERU LIBRE,,,"8,836,380",50.126%,VER LISTA
2,FUERZA POPULAR,,,"8,792,117",49.874%,VER LISTA
3,VOTOS EN BLANCO,,,"121,489",,
4,VOTOS NULOS,,,"1,106,816",,
...,...,...,...,...,...,...
140,ALIANZA UNIDAD DE IZQUIERDA,,"116,890",2.835%,VER LISTA,None
141,UNIDAD DEMOCRATICO POPULAR,,"98,452",2.387%,VER LISTA,None
142,FRENTE NACIONAL DE TRABAJADORES Y CAMPESINOS,,"81,647",1.980%,VER LISTA,None
143,FRENTE OBRERO CAMPESINO ESTUDIANTIL Y POPULAR,,"60,853",1.476%,VER LISTA,None


In [8]:
# See the NaN values
nulos = df.isnull().sum()
nulos

Organización Política       17
Símbolo                     17
Plan de Gobierno            17
Total Votos                 17
Porcentaje Votos Válidos    17
Lista de Candidatos         89
dtype: int64

In [9]:
# Delete NaN values
df.dropna(inplace=True)

In [10]:
# Check that there aren't NaN values
nulos_2 = df.isnull().sum()
nulos_2

Organización Política       0
Símbolo                     0
Plan de Gobierno            0
Total Votos                 0
Porcentaje Votos Válidos    0
Lista de Candidatos         0
dtype: int64

In [11]:
# Remove the columns we don't want
columns_to_exclude = ['Símbolo', 'Plan de Gobierno', 'Porcentaje Votos Válidos', 'Lista de Candidatos']
df = df.drop(columns=columns_to_exclude, errors='ignore')
df

,Organización Política,Total Votos
1,PARTIDO POLITICO NACIONAL PERU LIBRE,"8,836,380"
2,FUERZA POPULAR,"8,792,117"
3,VOTOS EN BLANCO,"121,489"
4,VOTOS NULOS,"1,106,816"
6,PARTIDO POLITICO NACIONAL PERU LIBRE,"2,724,752"
7,FUERZA POPULAR,"1,930,762"
8,RENOVACION POPULAR,"1,692,279"
9,AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL,"1,674,201"
10,ACCION POPULAR,"1,306,288"
11,JUNTOS POR EL PERU,"1,132,577"


In [12]:
# Change the directory
os.chdir(r'C:\Users\ittito\Documents\QLAB_Summer_Python\assignments\assignment_5')

In [13]:
# Export the data to an excel
df.to_excel("Elecciones_presidenciales.xlsx",index=False)